In [1]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objs as go
import pandas as pd
import backtrader as bt
import yfinance as yf
from datetime import datetime

C:\Users\User\AppData\Local\Temp\ipykernel_20244\3209428727.py:2: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
C:\Users\User\AppData\Local\Temp\ipykernel_20244\3209428727.py:3: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


In [2]:
class LazyGoldStrategy(bt.Strategy):
    def __init__(self):
        self.spy = self.datas[0]
        self.gld = self.datas[1]
        self.val_start = self.broker.get_cash()

    def next(self):
        if self.data.datetime.date().month == 12 or self.data.datetime.date().month == 1 or self.data.datetime.date().month == 2 or self.data.datetime.date().month == 8:
            self.buy(self.gld)
        else:
            self.buy(self.spy)
    
    def stop(self):
        self.roi = (self.broker.get_value() / self.val_start) - 1.0

In [3]:
class SixtyFortyStrategy(bt.Strategy):
    def __init__(self):
        self.spy = self.datas[0]
        self.agg = self.datas[1]
        self.val_start = self.broker.get_cash()

    def next(self):
        if self.data.datetime.date().year != self.data.datetime.date(-1).year:
            self.order_target_percent(self.spy, target=0.6)
            self.order_target_percent(self.agg, target=0.4)

    def stop(self):
        self.roi = (self.broker.get_value() / self.val_start) - 1.0

In [4]:
class BuyAndHoldStrategy(bt.Strategy):
    def start(self):
        self.val_start = self.broker.get_cash()

    def nextstart(self):
        self.buy(size=(self.broker.get_cash() / self.data))

    def stop(self):
        self.roi = (self.broker.get_value() / self.val_start) - 1.0


In [5]:
# Load Data
spy_data = yf.download('SPY', start='2011-05-05', end='2019-10-03', auto_adjust=True)
gld_data = yf.download('GLD', start='2011-05-05', end='2019-10-03', auto_adjust=True)
agg_data = yf.download('AGG', start='2011-05-05', end='2019-10-03', auto_adjust=True)

# Create the data feeds for backtrader
data_spy = bt.feeds.PandasData(dataname=spy_data)
data_gld = bt.feeds.PandasData(dataname=gld_data)
data_agg = bt.feeds.PandasData(dataname=agg_data)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [10]:
# Create a separate cerebro instance for each strategy
cerebro1 = bt.Cerebro()
cerebro2 = bt.Cerebro()
cerebro3 = bt.Cerebro()

# Add the data feeds to each cerebro instance
cerebro1.adddata(data_spy)
cerebro1.adddata(data_gld)

cerebro2.adddata(data_spy)
cerebro2.adddata(data_agg)

cerebro3.adddata(data_spy)

# Add the corresponding strategy to each cerebro instance
cerebro1.addstrategy(LazyGoldStrategy)
cerebro2.addstrategy(SixtyFortyStrategy)
cerebro3.addstrategy(BuyAndHoldStrategy)

# Set the initial cash for each cerebro instance
cerebro1.broker.setcash(10000.0)
cerebro2.broker.setcash(10000.0)
cerebro3.broker.setcash(10000.0)

# Add analyzers to each cerebro instance
cerebro1.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe_ratio')
cerebro1.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')

cerebro2.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe_ratio')
cerebro2.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')

cerebro3.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe_ratio')
cerebro3.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')

# Run the backtests
lazy_strategy = cerebro1.run()
sixty_forty_strategy = cerebro2.run()
buy_and_hold_strategy = cerebro3.run()

In [12]:
# Get the strategies
lazy_strategy = results1[0]
sixty_forty_strategy = results2[0]
buy_and_hold_strategy = results3[0]

# Now you can get the roi and the analyzers
lazy_roi = lazy_strategy.roi
lazy_sharpe_ratio = lazy_strategy.analyzers.sharpe_ratio.get_analysis()['sharperatio']
lazy_drawdown = lazy_strategy.analyzers.drawdown.get_analysis()['drawdown']

sixty_forty_roi = sixty_forty_strategy.roi
sixty_forty_sharpe_ratio = sixty_forty_strategy.analyzers.sharpe_ratio.get_analysis()['sharperatio']
sixty_forty_drawdown = sixty_forty_strategy.analyzers.drawdown.get_analysis()['drawdown']

buy_and_hold_roi = buy_and_hold_strategy.roi
buy_and_hold_sharpe_ratio = buy_and_hold_strategy.analyzers.sharpe_ratio.get_analysis()['sharperatio']
buy_and_hold_drawdown = buy_and_hold_strategy.analyzers.drawdown.get_analysis()['drawdown']


In [13]:
# Prepare data for the metrics table
metrics_data = pd.DataFrame(
    {
        'Strategy': ['Lazy Gold Strategy', '60/40 Strategy', 'Buy and Hold Strategy'],
        'ROI': [lazy_roi, sixty_forty_roi, buy_and_hold_roi],
        'Sharpe Ratio': [lazy_sharpe_ratio, sixty_forty_sharpe_ratio, buy_and_hold_sharpe_ratio],
        'Drawdown': [lazy_drawdown, sixty_forty_drawdown, buy_and_hold_drawdown]
    }
)

In [14]:
metrics_data

,Strategy,ROI,Sharpe Ratio,Drawdown
0,Lazy Gold Strategy,0.952965,0.795272,3.372174
1,60/40 Strategy,0.961697,1.005868,2.110878
2,Buy and Hold Strategy,0.000000,NaN,0.000000
